# Test Models

In [2]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the models
loaded_10_model = tf.keras.models.load_model('my_10_epoch_model')
loaded_20_model = tf.keras.models.load_model('my_20_epoch_model')
loaded_30_model = tf.keras.models.load_model('my_30_epoch_model')

IMG_SIZE = 422

def get_score_from_bin(bin_num):
    return bin_num*100 + 260  # Since we started binning from 260

def get_score_from_filename(filename):
    return int(filename.split('_')[0])  # Assuming score is at the beginning of the filename before '_'

def test_models(models, img_dir, num_images=100):
    total_losses = []

    for model in models:
        total_loss = 0

        for _ in range(num_images):
            print("running...", end="\r")  # Print running status

            random_filename = random.choice([
                x for x in os.listdir(img_dir)
                if os.path.isfile(os.path.join(img_dir, x))
            ])

            # Load the image
            img = load_img(os.path.join(img_dir, random_filename), target_size=(IMG_SIZE, IMG_SIZE))

            # Convert the image to a numpy array and reshape it
            img_array = img_to_array(img)
            img_array = img_array.reshape(1, IMG_SIZE, IMG_SIZE, 3)

            prediction = model.predict(img_array, verbose=0)

            pred_score = get_score_from_bin(prediction.argmax())
            true_score = get_score_from_filename(random_filename)

            loss = abs(pred_score - true_score)
            total_loss += loss

        avg_loss = total_loss / num_images
        total_losses.append(avg_loss)

    return total_losses

dir_name = 'combined_scored_images'
models = [loaded_10_model, loaded_20_model, loaded_30_model]

# Test the models
losses = test_models(models, dir_name, num_images=1000)

# Print the losses
for i, loss in enumerate(losses):
    print(f"Average loss for {10*(i+1)} epoch model: {loss}")


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x1964b7ba0>

In [ ]:
import matplotlib.pyplot as plt

# Create bar chart
plt.bar(['10 epochs', '20 epochs', '30 epochs'], losses)

# Add title and labels
plt.title('Average Loss for Different Models')
plt.xlabel('Model')
plt.ylabel('Average Loss')

# Show the plot
plt.show()
